In [0]:
import org.apache.spark.sql.types._
val myDataSchema = StructType(
Array(
StructField("id_persona", DecimalType(26, 0), true), 
StructField("anio", IntegerType, true), 
StructField("mes", IntegerType, true), 
StructField("provincia", IntegerType, true), 
StructField("canton", IntegerType, true), 
StructField("area", StringType, true), 
StructField("genero", StringType, true), 
StructField("edad", IntegerType, true), 
StructField("estado_civil", StringType, true), 
StructField("nivel_de_instruccion", StringType, true), 
StructField("etnia", StringType, true), 
StructField("ingreso_laboral", IntegerType, true), 
StructField("condicion_actividad", StringType, true), 
StructField("sectorizacion", StringType, true), 
StructField("grupo_ocupacion", StringType, true), 
StructField("rama_actividad", StringType, true), 
StructField("factor_expansion", DoubleType, true)
));//le damos al archivo el esquema establecido anteriormente
val data = spark
.read
.schema(myDataSchema)
.option("header", "true")
.option("delimiter", "\t")
.csv("Datos_ENEMDU_PEA_v2.csv")

#### Que porcentaje de indigenas fueron encuestados ?

In [2]:
val indg = data.where($"etnia" === "1 - Indígena")
print(f"${(indg.count * 100)/data.count.toDouble}%.2f%% De Indígenas encuestados")

## ¿Cuál es la clasificacion de indigenas con respecto a sus condicion laboral (condicion_actividad)?


In [4]:
//ojo
z.show(f"${(indg.where($"condicion_actividad" === "1 - Empleo Adecuado/Pleno").count * 100)/indg.count.toDouble}%.2f%% Empleo Adecuado\n${(indg.where($"condicion_actividad" === "2 - Subempleo por insuficiencia de tiempo de trabajo").count * 100)/indg.count.toDouble}%.2f%% Subempleo por insuficiencia de trabajo\n${(indg.where($"condicion_actividad" === "4 - Otro empleo no pleno").count * 100)/indg.count.toDouble}%.2f%% Otro empleo no pleno\n${(indg.where($"condicion_actividad" === "5 - Empleo no remunerado").count * 100)/indg.count.toDouble}%.2f%% Empleo no remunerado\n${(indg.where($"condicion_actividad" === "6 - Empleo no clasificado").count * 100)/indg.count.toDouble}%.2f%% Empleo no clasificado\n${(indg.where($"condicion_actividad" === "7 - Desempleo abierto").count * 100)/indg.count.toDouble}%.2f%% Desempleo Abierto\n${(indg.where($"condicion_actividad" === "8 - Desempleo oculto").count * 100)/indg.count.toDouble}%.2f%% Desempleo Oculto")


## ¿Cuál es la cantidad de Indígenas que estan con desempleo abierto(busqueda activamente de empleo durante el mes anterior a la encuesta sin exito)?


In [6]:
indg.where($"condicion_actividad" === "7 - Desempleo abierto").count

## ¿Cuál es el nivel de educación que tienen los Indígenas desempleados abiertamente?

In [8]:
val indgEduc =  indg.where($"condicion_actividad" === "7 - Desempleo abierto") 
z.show(	indgEduc.groupBy("nivel_de_instruccion").count().as("cantidad").sort(desc("count")))

## ¿Puede una persona Indigena con desempleo abierto, tener un ingreso laboral diferente de 0? ¿No es esto contradictorio? Compruebelo.

In [10]:
indgEduc.where($"ingreso_laboral" > 0)
indgEduc.where($"ingreso_laboral" > 0).count


### ¿Cómo es posible que una persona con un "desempleo abierto" tenga un ingreso laboral que no sea 0?
Se puede considerar varios posibles errores:
* Primero: La sentencia este mal escrita o con un error, lo cual es posible, pero se descarta al usar otra herramienta para comprobar 
su veracidad (excel) en el cual demuestra usando filtros que existen encuestados indigenas, con desempleo oculto que tienen 
un ingreso laboral mayor a 0.

* Segundo: El archivo de origen vino con defectos, otra opcion posible, pero tambien descartada cuando leemos los significados de cada
columna.

* Posible Respuesta: La pagina web desde donde se proveyeron los datos, ofrece información acerca de que es el desempleo oculto y que 
considera el mismo. Siendo asi el significado de esta variable "Personas sin empleo, que no estuvieron empleados en la semana pasada y que buscaron trabajo e
hicieron gestiones concretas para conseguir empleo o para establecer algún negocio en las cuatro semanas anteriores a la entrevista."

Se puede considerar como verdadera esta sentencia ya que, podemos deducir que el ingreso laboral que decretaron estas personas en la encuesta
pertenece al ingreso laboral que recibian una semana antes de la encuesta, cuando aun estaban empleados.

### ¿Cuál es el valor máximo y minimo que gana un indígena que tiene el nivel mas alto de estudios (Post-grado)?

In [12]:
val indgEducPost = (indg.where($"nivel_de_instruccion" === "10 - Post-grado"))
indgEducPost.select("ingreso_laboral").summary("max", "min").show()

## ¿Existe una diferencia de salario que un indigena(post-grado) recibe de ingreso laboral promedio, con respecto al valor de ingreso laboral promedio de un ecuatoriano meztizo(post-grado)?

In [14]:
indgEducPost.select("ingreso_laboral").summary().show()

#### ¿Existen valores nulos?


In [16]:
indgEducPost.select("ingreso_laboral").groupBy("ingreso_laboral").count().sort(desc("count")).show(5)

#### Limpieza de Nulos

In [18]:
val postIndg = indgEducPost.select("ingreso_laboral").where($"ingreso_laboral".isNotNull)

#### Busqueda de acotas

In [20]:
val cantValoresEnDifRangos =  scala.collection.mutable.ListBuffer[Long]()
		val minValue = 0.0
		val maxValue = 5264
		val bins = 5.0
		val range = (maxValue - minValue)/bins
		var minCounter = minValue
		var maxCounter = range
		while (minCounter < maxValue){
		  val valoresEnUnRango = postIndg.where($"ingreso_laboral".between(minCounter,maxCounter))
		  cantValoresEnDifRangos.+=(valoresEnUnRango.count())
		  minCounter = maxCounter
		  maxCounter = maxCounter + range
		}

#### Resultados de rangos

In [22]:
println("Valores en diferentes rangos: ")
cantValoresEnDifRangos.foreach(println)
		/* 77 rango 0 - 1053
		49 rango 1053 - 2106
		23 rango 2106 - 3158
		8  rango 3158 - 4211 
		2  rango 4211 - 5264*/

#### Promedio

In [24]:
val prom = postIndg.select(mean("ingreso_laboral")).first()(0).asInstanceOf[Double]

#### Desviacion estandar

In [26]:
val desviacion = postIndg.select(stddev("ingreso_laboral")).first()(0).asInstanceOf[Double]

#### Limite Inferior

#### Limite Superior

In [29]:
val limInf = prom - 3 * desviacion

In [30]:
val limSup = prom + 3 * desviacion

In [31]:
val indgPostSinAcotas = postIndg.where($"ingreso_laboral" > limInf && $"ingreso_laboral" < limSup)

#### sin acotas

#### con acotas

In [34]:
indgPostSinAcotas.select("ingreso_laboral").summary().show	

In [35]:
indgEducPost.select("ingreso_laboral").summary().show()

In [36]:
val mezt = data.where($"etnia" === "6 - Mestizo")
val meztEducPost = mezt.where($"nivel_de_instruccion" === "10 - Post-grado")

#### Summary de Meztizos con Postgrado (Con acotas)

In [38]:
meztEducPost.select("ingreso_laboral").summary().show()

#### ¿Existen valores nulos?

In [40]:
meztEducPost.select("ingreso_laboral").groupBy("ingreso_laboral").count().sort(desc("count")).show(5)

#### Limpieza de Nulos

In [42]:
val postMezt = meztEducPost.select("ingreso_laboral").where($"ingreso_laboral".isNotNull)

 
#### Busqueda de acotas

In [44]:
val cantValoresEnDifRangos =  scala.collection.mutable.ListBuffer[Long]()
val minValue = 0.0
val maxValue = 30000
val bins = 5.0
val range = (maxValue - minValue)/bins
var minCounter = minValue
var maxCounter = range
while (minCounter < maxValue){
val valoresEnUnRango = postMezt.where($"ingreso_laboral".between(minCounter,maxCounter))
cantValoresEnDifRangos.+=(valoresEnUnRango.count())
minCounter = maxCounter
maxCounter = maxCounter + range
}

#### Resultados de rangos

In [46]:
println("Valores en diferentes rangos: ")
cantValoresEnDifRangos.foreach(println)

#### Promedio

In [48]:
val prom = postMezt.select(mean("ingreso_laboral")).first()(0).asInstanceOf[Double]

#### Desviacion estandar

In [50]:
val desviacion = postMezt.select(stddev("ingreso_laboral")).first()(0).asInstanceOf[Double]

#### Limite Inferior

#### Limite Superior

In [53]:
val limInf = prom - 3 * desviacion

In [54]:
val limSup = prom + 3 * desviacion

#### Meztizos con Postgrado (Sin acotas)

In [56]:
val meztPostSinAcotas = postMezt.where($"ingreso_laboral" > limInf && $"ingreso_laboral" < limSup)

Sin acotas

Con acotas

In [59]:
meztPostSinAcotas.select("ingreso_laboral").summary().show		

In [60]:
meztEducPost.select("ingreso_laboral").summary().show()

 
## Boxplots

Indigenas

Mestizos

In [64]:
val cuartiles = postIndg.stat.approxQuantile("ingreso_laboral", Array (0.25, 0.75), 0.0)
val q1 = cuartiles(0)
val q3 = cuartiles(1)
val iQR = q3 -q1
val inferiorIQR = q1 -1.5 * iQR
val superiorIQR = q3 + 1.5 * iQR
val indgSinAcotasIQR = postIndg.where($"ingreso_laboral" > inferiorIQR && $"ingreso_laboral" < superiorIQR)
indgSinAcotasIQR.select("ingreso_laboral").summary().show

In [65]:
val cuartiles = postMezt.stat.approxQuantile("ingreso_laboral", Array (0.25, 0.75), 0.0)
val q1 = cuartiles(0)
val q3 = cuartiles(1)
val iQR = q3 -q1
val inferiorIQR = q1 -1.5 * iQR
val superiorIQR = q3 + 1.5 * iQR
val meztSinAcotasIQR = postMezt.where($"ingreso_laboral" > inferiorIQR && $"ingreso_laboral" < superiorIQR)
meztSinAcotasIQR.select("ingreso_laboral").summary().show

Considerando los resultados de los 2 algoritmos, se puede deducir que en efecto, existe una diferencia entre lo que gana un indigena (post-grado) en promedio,
con lo que gana un meztizo de similares caracteristicas, pero realmente no es una diferencia significativa, es un valor encontrado en un rango entre 130 -170.
Vease:

Algoritmo 1 (salario promedio):

indigena =  1387.386075949367		meztizo = 1553.2442199775533

diferencia = | 1387.39 - 1553.24 | = |- 165.85 |
diferencia = 165.85

Algoritmo 2 (salario promedio):

indigena =  1291.5526315789473		meztizo = 1425.7911637173327

diferencia = | 1291.55 - 1425.8 | = |- 134.25 |
diferencia = 134.25*/

#### ¿Es posible que con el tiempo los niveles de empleo adecuado y desempleo en los indigenas hayan aumentado con respecto a los demas grupos poblacionales? ¿Es posible demostrar como ha variado su condicion laboral con el paso de los años?

In [68]:
z.show(indg.stat.crosstab("condicion_actividad", "anio").orderBy("condicion_actividad_anio"))

¿Que hay con respecto a las personas que no son Indigenas? ¿Se puede observar una similaridad en cuanto a la estadistica?

In [70]:
val noIndg = data.where($"etnia" =!= "1 - Indígena")
z.show(noIndg.stat.crosstab("condicion_actividad", "anio").orderBy("condicion_actividad_anio"))

Los datos demuestran que no ha habido algun tipo de discriminacion, todos los grupos poblacionales han demostrado que han tendido a la baja con respecto al empleo adecuado en el ecuador	*/

#### Cual es el porcentaje de ingreso laboral que un indigena genera?


In [73]:
print(f"${(indg.count * 100)/data.count.toDouble}%.2f%% De Indígenas encuestados")

#### De las personas indigenas ¿Es posible que hayan sido las mujeres las mas que menor valor de ingreso han obtenido? ¿Existe alguna diferencia significativa entre el porcentaje de ingresos laborales que generaron las mujeres con respecto a los hombres?

In [75]:
	z.show(indg.groupBy("anio").pivot("genero").agg( round((sum("ingreso_laboral")*100)/13925210) ).orderBy("anio"))

## Variables a comparar
En el Ecuador historicamente los indigenas han sido el grupo mas vulnerable desde la conquista de los espanoles,desde entonces este grupo etnico ha sido discriminado por los demas grupos, relegandolos a una posicion desfavorable con respecto a otros grupos etnicos, al otro lado de la balanza tenemos a los mestizos quienes en la actualidad son el grupo etnico comun en el pais y muchos lo consideran como el privilegiado 

Actualmente existe la creencia de que estas diferencias ya quedaron en el pasado y que actualmente no existe gran diferencia entre estos grupos, por lo que buscaremos datos acerca del deseempleo y la evolucion de este en ambos grupos sociales para despues compararlos y llegar a una conclusion 

#### Porcentaje de desempleo indigena en el area urbana y rural 
Para estas consultas nos centraremos en las siguientes columnas:
* area: La columna describe el area donde se encuentra viviendo la persona (rural o urbana)
* condicion_activada: La columna describe como estasu condicion laboral actualmente es decir si se encuentra empleado o no


In [78]:
val indgDes = indg.select("area","condicion_actividad").where(column("condicion_actividad")==="7 - Desempleo abierto").count
val des = (indg.select("area","condicion_actividad").where(column("condicion_actividad")==="7 - Desempleo abierto").groupBy("area").count)
z.show(des.select(des("area"),(des("count")/indgDes *100).as("porcentaje deseempleo")))

In [79]:
//mestizos
val des = (mezt.select("area","condicion_actividad").where(column("condicion_actividad")==="7 - Desempleo abierto").groupBy("area").count)
z.show(des.select(des("area"),(des("count")/622776 *100).as("porcentaje deseempleo")))

In [80]:
//se muestra la media de ingresos por cada grupo de ocupacion
z.show(indg.select("grupo_ocupacion","ingreso_laboral").groupBy("grupo_ocupacion").avg("ingreso_laboral").select($"grupo_ocupacion",$"avg(ingreso_laboral)".as("media de ingresos")))

In [81]:
//meztizos
z.show(mezt.select("grupo_ocupacion","ingreso_laboral").groupBy("grupo_ocupacion").avg("ingreso_laboral").select($"grupo_ocupacion",$"avg(ingreso_laboral)".as("media de ingresos")))

In [82]:
//se muestra la media de ingresos segun la zona  
z.show(indg.select("area","ingreso_laboral").groupBy("area").avg("ingreso_laboral").select($"area",$"avg(ingreso_laboral)".as("media de ingresos")))

In [83]:
//meztizos
z.show(mezt.select("area","ingreso_laboral").groupBy("area").avg("ingreso_laboral").select($"area",$"avg(ingreso_laboral)".as("media de ingresos")))

In [84]:
//se muestra la media de ingresos segun el nivel de instruccion 
z.show(indg.select("nivel_de_instruccion","ingreso_laboral").groupBy("nivel_de_instruccion").avg("ingreso_laboral").orderBy("nivel_de_instruccion").select($"nivel_de_instruccion",$"avg(ingreso_laboral)".as("media de ingresos")))

In [85]:
//meztizos
z.show(mezt.select("nivel_de_instruccion","ingreso_laboral").groupBy("nivel_de_instruccion").avg("ingreso_laboral").orderBy("nivel_de_instruccion").select($"nivel_de_instruccion",$"avg(ingreso_laboral)".as("media de ingresos")))

In [86]:
//se muestra el deseempleo abierto segun el nivel de educacion
z.show(indg.select("nivel_de_instruccion","condicion_actividad").where(column("condicion_actividad")==="7 - Desempleo abierto").groupBy("nivel_de_instruccion").count.orderBy(col("nivel_de_instruccion").desc))
//indg.stat.crosstab("nivel_de_instruccion", "condicion_actividad").orderBy("nivel_de_instruccion_condicion_actividad").show

In [87]:
//meztizos
z.show(mezt.select("nivel_de_instruccion","condicion_actividad").where(column("condicion_actividad")==="7 - Desempleo abierto").groupBy("nivel_de_instruccion").count.orderBy(col("nivel_de_instruccion").desc))

In [88]:
//se muestra el deseempleo abierto segun el estado civil

z.show(indg.select($"estado_civil",$"condicion_actividad").where(column("condicion_actividad")==="7 - Desempleo abierto").groupBy("estado_civil").count.select($"estado_civil",
$"count"/622776 *100).select($"estado_civil",$"((count / 622776) * 100)".as("porcentaje deseempleo")))

In [89]:
//meztizos
z.show(mezt.select($"estado_civil",$"condicion_actividad").where(column("condicion_actividad")==="7 - Desempleo abierto").groupBy("estado_civil").count.select($"estado_civil",
$"count"/622776 *100).select($"estado_civil",$"((count / 622776) * 100)".as("porcentaje deseempleo")))

In [90]:
//como ha evolucionado el deseempleo en la poblacion indigena en este tiempo ?
val indg2 = indg.where(column("condicion_actividad")==="7 - Desempleo abierto")
z.show(indg2.stat.crosstab("anio", "genero").orderBy("anio_genero"))

In [91]:
//meztizo
val indg2 = mezt.where(column("condicion_actividad")==="7 - Desempleo abierto")
z.show(indg2.stat.crosstab("anio", "genero").orderBy("anio_genero"))

In [92]:
//como ha evolucionado el nivel de instruccion en el pueblo indigena a traves del tiempo 
//como se explican estos numeros
z.show(indg.groupBy("nivel_de_instruccion").pivot("anio").count)

In [93]:
//meztizo
z.show(mezt.groupBy("nivel_de_instruccion").pivot("anio").count)